# Uncertainty Estimates from Classifiers

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_circles
from helpers.plot_helpers import discrete_scatter

X, y = make_circles(noise=0.25, factor=0.5, random_state=1)

In [ ]:
discrete_scatter(X[:, 0], X[:, 1], y, markers='o')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# we rename the classes "blue" and "red" for illustration purposes
y_named = np.array(["blue", "red"])[y]

# we can call train_test_split with arbitrarily many arrays;
# all will be split in a consistent manner
X_train, X_test, y_train_named, y_test_named, y_train, y_test = train_test_split(X, y_named, y, random_state=0)

# build the gradient boosting model
gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train_named)

## The Decision Function

In [ ]:
print(f"X_test.shape: {X_test.shape}")
print(f"Decision function shape: {gbrt.decision_function(X_test).shape}")

In [ ]:
# show the first few entries of decision_function
gbrt.decision_function(X_test)[:6]

In [ ]:
gbrt.decision_function(X_test) > 0

In [ ]:
gbrt.predict(X_test)

In [ ]:
gbrt.classes_

In [ ]:
# make the boolean True/False into 0 and 1
greater_zero = (gbrt.decision_function(X_test) > 0).astype(int)

# use 0 and 1 as indices into classes_
pred = gbrt.classes_[greater_zero]

# pred is the same as the output of gbrt.predict
print(f"pred is equal to predictions: {np.all(pred == gbrt.predict(X_test))}")

In [ ]:
decision_function = gbrt.decision_function(X_test)

print(f"Decision function minimum: {np.min(decision_function):.2f} maximum: {np.max(decision_function):.2f}")

In [ ]:
from helpers import tools
from helpers import plot_helpers

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

tools.plot_2d_separator(gbrt, X, ax=axes[0], alpha=.4, fill=True, cm=plot_helpers.cm2)

scores_image = tools.plot_2d_scores(gbrt, X, ax=axes[1], alpha=.4, cm=plot_helpers.ReBl)


for ax in axes:
    # plot training and test points
    discrete_scatter(X_test[:, 0], X_test[:, 1], y_test, markers='^', ax=ax)
    discrete_scatter(X_train[:, 0], X_train[:, 1], y_train, markers='o', ax=ax)
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")
    
plt.rcParams['axes.grid'] = False
cbar = plt.colorbar(scores_image, ax=axes.tolist())
axes[0].legend(["Test class 0", "Test class 1", "Train class 0", "Train class 1"], ncol=4, loc=(.1, 1.1))
plt.show()

## Predicting Probabilities

In [ ]:
gbrt.predict_proba(X_test).shape

In [ ]:
# gbrt.predict_proba(X_test[:6])
gbrt.predict_proba(X_test[:6])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5)) 

tools.plot_2d_separator(gbrt, X, ax=axes[0], alpha=.4, fill=True, cm=plot_helpers.cm2)
scores_image = tools.plot_2d_scores(gbrt, X, ax=axes[1], alpha=.5, cm=plot_helpers.ReBl, function='predict_proba')

for ax in axes:
    # plot training and test points
    discrete_scatter(X_test[:, 0], X_test[:, 1], y_test, markers='^', ax=ax)
    discrete_scatter(X_train[:, 0], X_train[:, 1], y_train, markers='o', ax=ax)
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")

plt.rcParams['axes.grid'] = False
cbar = plt.colorbar(scores_image, ax=axes.tolist())
axes[0].legend(["Test class 0", "Test class 1", "Train class 0", "Train class 1"], ncol=4, loc=(.1, 1.1))
plt.show()

## Uncertainty in Multiclass Classification

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=42)

gbrt = GradientBoostingClassifier(learning_rate=0.01, random_state=0)
gbrt.fit(X_train, y_train)

In [ ]:
gbrt.decision_function(X_test).shape

In [ ]:
gbrt.decision_function(X_test)[:6, :]

In [ ]:
np.argmax(gbrt.decision_function(X_test), axis=1)

In [ ]:
gbrt.predict(X_test)

In [ ]:
#show the first few entries of predict_proba
print(f"Predicted probabilities:\n{gbrt.predict_proba(X_test)[:6]}")
# show that sums across rows are one
print(f"Sums: {gbrt.predict_proba(X_test)[:6].sum(axis=1)}")

In [ ]:
np.argmax(gbrt.predict_proba(X_test), axis=1)

In [ ]:
gbrt.predict(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)

# represent each target by its class name in the iris dataset
named_target = iris.target_names[y_train]

logreg.fit(X_train, named_target)

print(f"unique classes in training data: {logreg.classes_}")
print(f"predictions: {logreg.predict(X_test)[:10]}")

argmax_dec_func = np.argmax(logreg.decision_function(X_test), axis=1)
print(f"argmax of decision function: {argmax_dec_func[:10]}")
print(f"argmax combined with classes_: {logreg.classes_[argmax_dec_func][:10]}")